## Exercise to illustrate how the correlation time varies with the azimuth to the source and how the "stationary phase" contribution emerges in the stacked cross-correlation

## Main Python packages used: 

- [**NumPy**](https://numpy.org) : for mathematical functions

- [**Matplotlib**](https://matplotlib.org) : for plotting results

- [**SciPy**](https://github.com/obspy/obspy/wiki) : to compute [**cross-correlations**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate.html)

---



In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt


#------------------ plotting mode
%matplotlib widget
#----------------------------


plt.close('all')


#------------------------------------------------
# subroutines for generating impulsive sgnal and noise
#------------------------------------------------

#-------------------------- single Ricker pulse

def one_ricker(t,delay,f):
    a = np.pi*np.pi*f*f*(t-delay)**2
    return (1-2*a)*np.exp(-a)

#-------------------------- random signal as sum of many Ricker pulses
dt = 0.5
f = .07             # ricker frequency
n_rick = 25

trick = np.arange(-n_rick,n_rick)*dt
rick0 = one_ricker(trick,0,f)

def fast_random_rickers(t,nsig,delay,f,lam,nt,dt):
    a = 2*np.random.rand(nsig)-1
    r = np.random.rand(nsig)
    dts = -np.log(r)/lam
    ts = np.cumsum(dts)-250
    tmax = nt*dt
    drange = 1/f
    u = 0*t
    for i in range (0, nsig):
        delI = ts[i]+delay
        if delI > tmax:
            break
        dif_t = np.around((t-delI)/dt)
        i_dif_t = dif_t.astype(int)
        idx = (i_dif_t>-n_rick) & (i_dif_t<n_rick)
        ii = np.around((t[idx]-delI)/dt)
        iii = ii.astype(int)+n_rick
        u[idx] = u[idx] + a[i]*rick0[iii]
    return u


        

#------------------------------------------------
# defining parmeters

dist = 500
vel = 5

npt = 5000
nsig = 8000
lam = .3


#dt = 0.5
#f = .07             # ricker frequency

#------------------------------------------------
delay1 = 150;

#..............................

t = np.arange(npt)
t = dt*t

tcc = np.arange(-(npt-1),npt)
tcc = dt*tcc

ccsum = 0*tcc

#------------------------------------------------
# computing plotting individual cross-correlations
#..............................
plt.figure()

#..............................
for i in range (0, 180):
    ang = i*2
    if ( int(ang/30) == ang/30) :
        print(ang)
    angr = np.pi*ang/180
    delay2 = delay1 + dist*np.sin(angr)/vel

#--------------------------------- deterministic (impulsive) source in time
#    sig1 = one_ricker(t,delay1,f)
#    sig2 = one_ricker(t,delay2,f)

#--------------------------------- random (continuous) source in time
    seed = np.random.randint(100000,size=1)
    
    np.random.seed(seed)
    sig1 = fast_random_rickers(t,nsig,delay1,f,lam,npt,dt)
    np.random.seed(seed)
    sig2 = fast_random_rickers(t,nsig,delay2,f,lam,npt,dt)
    

    cc = signal.correlate(sig2,sig1,'full')/np.sqrt(np.sum(sig2**2)*np.sum(sig1**2))

    plt.plot(tcc,ang+3.6*cc,'k',linewidth=.5)
    
    ccsum = ccsum + cc

plt.xlim(-200,200)

plt.title('cross-correlations from individual sources')
plt.xlabel('correlation time (s)')
plt.ylabel('azimuth (deg)')

plt.show()



#------------------------------------------------
# plotting stacked cross-correlation
#..............................

plt.figure()

plt.plot(tcc,ccsum,'k')
plt.xlim(-200,200)

plt.title('stacked cross-correlation')
plt.xlabel('correlation time (s)')

plt.show()


#------------------------------------------------
# plotting source-station geometry
#..............................

plt.figure()

plt.axes().set_aspect('equal')


y = 1250
x = 1500
plt.plot(x,y,'r^')
x = 1000
plt.plot(x,y,'b^')


#..............................
for i in range (0, 180):
    ang = i*2
    angr = np.pi*ang/180
    x = 1250 + 750*np.sin(angr)
    y = 1250 + 750*np.cos(angr)
    plt.plot(x,y,'k*')
    
plt.ylim(0,2500)
plt.xlim(0,2500)

plt.axhline(y=1250,color='k',linestyle=':',linewidth=1)
plt.axvline(x=1250,color='k',linestyle=':',linewidth=1)

plt.title('sources (star) and receivers (triangle) positions')
plt.xlabel('distance (km)')
plt.ylabel('distance (km)')


plt.show()



